In [1]:
from scipy.stats.mstats import mquantiles  # I think this is stats with masked arrays.

In [3]:
import numpy as np

In [10]:
data = np.array([[   6.,    7.,    1.],
                 [  47.,   15.,    2.],
                 [  49.,   36.,    3.],
                 [  15.,   39.,    4.],
                 [  42.,   40., -999.],
                 [  41.,   41., -999.],
                 [   7., np.nan, -999.],
                 [  39., -999., -999.],
                 [  43., -999., -999.],
                 [  40., -999., -999.],
                 [  36., -999., -999.]])

ma_data = np.ma.masked_invalid(data)
ma_data = np.ma.masked_where(ma_data == -999., ma_data)
print(ma_data)
print(mquantiles(ma_data, axis=0, limit=(0, 50)))

[[6.0 7.0 1.0]
 [47.0 15.0 2.0]
 [49.0 36.0 3.0]
 [15.0 39.0 4.0]
 [42.0 40.0 --]
 [41.0 41.0 --]
 [7.0 -- --]
 [39.0 -- --]
 [43.0 -- --]
 [40.0 -- --]
 [36.0 -- --]]
[[19.2  14.6   1.45]
 [40.   37.5   2.5 ]
 [42.8  40.05  3.55]]


In [ ]:
[[19.2  14.6   1.45]
 [40.   37.5   2.5 ]
 [42.8  40.05  3.55]]

In [11]:
import xarray as xr

In [12]:
from pathlib import Path

In [13]:
stem = Path("/project/amp/brianpm/TemperatureExtremes/Regridded")
ds = xr.open_dataset(stem/"f.e13.FAMIPC5CN.ne30_ne30.beta17.t3.cam.h1.TREFMXAV.19650101-20051231.regrid.nc", decode_times=False)

In [17]:
tb = ds['time_bnds']
ds['time'].values = tb.mean(dim='nbnd')
ds = xr.decode_cf(ds)
print(ds)

<xarray.Dataset>
Dimensions:       (ilev: 31, lat: 360, lev: 30, lon: 720, nbnd: 2, time: 14966)
Coordinates:
  * lat           (lat) float64 89.75 89.25 88.75 88.25 ... -88.75 -89.25 -89.75
  * lon           (lon) float64 0.25 0.75 1.25 1.75 ... 358.2 358.8 359.2 359.8
  * ilev          (ilev) float64 2.255 5.032 10.16 18.56 ... 967.5 985.1 1e+03
  * lev           (lev) float64 3.643 7.595 14.36 24.61 ... 957.5 976.3 992.6
  * time          (time) object 1965-01-01 00:00:00 ... 2005-12-31 12:00:00
Dimensions without coordinates: nbnd
Data variables:
    lat_bnds      (lat, nbnd) float64 ...
    lon_bnds      (lon, nbnd) float64 ...
    gw            (lat) float64 ...
    area          (lat, lon) float64 ...
    P0            float64 ...
    TREFMXAV      (time, lat, lon) float32 ...
    ch4vmr        (time) float64 ...
    co2vmr        (time) float64 ...
    date          (time) int32 ...
    date_written  (time) |S8 ...
    datesec       (time) int32 ...
    f11vmr        (time) flo

In [16]:
ds['time']

<xarray.DataArray 'time' (time: 14966)>
array([0.00000e+00, 5.00000e-01, 1.50000e+00, ..., 1.49625e+04, 1.49635e+04,
       1.49645e+04])
Coordinates:
  * time     (time) float64 0.0 0.5 1.5 2.5 ... 1.496e+04 1.496e+04 1.496e+04
Attributes:
    long_name:  time
    units:      days since 1965-01-01 00:00:00
    calendar:   noleap
    bounds:     time_bnds

In [18]:
sacramento = (38. + 33/60,  360. - (121. + 28/60))
sac_tmax = ds['TREFMXAV'].sel(lat=sacramento[0], lon=sacramento[1], method='nearest')

In [19]:
sac_tmax

<xarray.DataArray 'TREFMXAV' (time: 14966)>
array([282.16833, 282.16833, 278.39264, ..., 282.98932, 284.37402, 282.28073],
      dtype=float32)
Coordinates:
    lat      float64 38.75
    lon      float64 238.8
  * time     (time) object 1965-01-01 00:00:00 ... 2005-12-31 12:00:00
Attributes:
    units:          K
    long_name:      Average of TREFHT daily maximum
    cell_methods:   time: mean
    cell_measures:  area: area

In [20]:
np.any(sac_tmax == np.nan)

<xarray.DataArray 'TREFMXAV' ()>
array(False)
Coordinates:
    lat      float64 38.75
    lon      float64 238.8

In [21]:
np.mean(sac_tmax)

<xarray.DataArray 'TREFMXAV' ()>
array(294.1676, dtype=float32)
Coordinates:
    lat      float64 38.75
    lon      float64 238.8

In [23]:
%%time
s = 0
for i in range(14966):
    s += 1
print(s)    

14966
CPU times: user 2.95 ms, sys: 912 µs, total: 3.86 ms
Wall time: 3.72 ms


In [30]:
%%time
print(sac_tmax.sum())

<xarray.DataArray 'TREFMXAV' ()>
array(4402512.5, dtype=float32)
Coordinates:
    lat      float64 38.75
    lon      float64 238.8
CPU times: user 40 s, sys: 1.2 s, total: 41.2 s
Wall time: 41.2 s


In [25]:
sample = np.ones((14966,))

In [28]:
sample.shape == sac_tmax.shape

True

In [29]:
%%time
print(sample.sum())

14966.0
CPU times: user 527 µs, sys: 37 µs, total: 564 µs
Wall time: 349 µs


In [31]:
%%time
tmx = sac_tmax.values

CPU times: user 40 s, sys: 1.16 s, total: 41.2 s
Wall time: 41.2 s


In [32]:
%%time
print(tmx.sum())

4402512.5
CPU times: user 264 µs, sys: 0 ns, total: 264 µs
Wall time: 195 µs


In [42]:
%%time
print(mquantiles(tmx, prob=[.05, .25, .50, .75, .90, .95, .99], limit=(200, 400)))

[280.08330597 286.30161896 293.64672852 302.42931671 306.71816833
 308.57774902 311.83853088]
CPU times: user 5.03 ms, sys: 59 µs, total: 5.08 ms
Wall time: 3.99 ms


In [43]:
%%time
print(np.nanquantile(tmx, [.05, .25, .50, .75,.90, .95, .99]))

[280.08447266 286.30200958 293.64672852 302.4272995  306.71812439
 308.57662964 311.81681824]
CPU times: user 1.61 ms, sys: 1.01 ms, total: 2.62 ms
Wall time: 1.62 ms
